In [1]:
# install bitcoin lib
!pip3 install bitcoin-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00
  Created wheel for python-bitcoinrpc: filename=python_bitcoinrpc-1.0-py3-none-any.whl size=14240 sha256=59380392c287db234878fa80a97d843f6854a20e96c05658d8ab17ae6a991dfc
  Stored in directory: /root/.cache/pip/wheels/58/8b/b6/bebc1e5b1eec4ee482d7eb8ea17fd133ce8db6c23d0c8b066b
Successfully built python-bitcoinrpc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.3 MB/s eta 0:00:00


# Task 1: Basic script execution
- The sender's address:
  - Private key: cTcXHR9mpY3Yh1XwQaGrrFRRpYR1XqPGNTb2PjMpPQ84kkafLyRN
  - Public key: 032f0d9e099d17b67c4bf4aaf4704ab99693177d26c7e4c598705e442836340327
  - Bitcoin address: mr3q9zULfWJktFDZMgpkTCa5ra9vJDdcJ4

- The recipient's address:
  - Private key: cS6t9sbsL9cTPwpdK1oX3FazLeTJBGwfXftVgbggNNha8QE6YbAK
  - Public key: 022752417ad3c5aa69b0e1b19cbd388d86dfdbf79a102235741e60665d13a01786
  - Bitcoin address: mtEKiuN1sf5sENSsCHcfAdQhJXsUxGj1vr

- The previous transaction identifier (txid): cc7fa05421d61b7a8948cc0e1115b15f2b8692f3fbec11d5b0ead711343d8c01

In [2]:
import os
from bitcoinutils.setup import setup
from bitcoinutils.keys import P2pkhAddress, PrivateKey, PublicKey

In [ ]:
#setup the network
setup('testnet')

# Generate a random private key
#private_key = PrivateKey.from_bytes(os.urandom(32))
private_key = PrivateKey.from_wif("cTcXHR9mpY3Yh1XwQaGrrFRRpYR1XqPGNTb2PjMpPQ84kkafLyRN")

# get the public key
public_key = private_key.get_public_key()

# get address from public key
address = public_key.get_address()

# print address and hash160
print("Private Key:", private_key.to_wif())
print("Public Key:", public_key.to_hex())
print("Bitcoin Address:", address.to_string())


Private Key: cTcXHR9mpY3Yh1XwQaGrrFRRpYR1XqPGNTb2PjMpPQ84kkafLyRN
Public Key: 032f0d9e099d17b67c4bf4aaf4704ab99693177d26c7e4c598705e442836340327
Bitcoin Address: mr3q9zULfWJktFDZMgpkTCa5ra9vJDdcJ4


In [3]:
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.script import Script

In [ ]:
# create transaction input from tx id of UTXO
txin = TxInput(
    "cc7fa05421d61b7a8948cc0e1115b15f2b8692f3fbec11d5b0ead711343d8c01", 0
)

# create a transaction output to the generated address
to_addr = P2pkhAddress("mtEKiuN1sf5sENSsCHcfAdQhJXsUxGj1vr")
txout = TxOutput(
        to_satoshis(0.001),
        Script(
            ["OP_DUP", "OP_HASH160", to_addr.to_hash160(), "OP_EQUALVERIFY", "OP_CHECKSIG"]
        ),
    )
# create the transaction
tx = Transaction([txin], [txout])
print("\nRaw unsigned transaction:\n" + tx.serialize())

# Sign the transaction
sig = private_key.sign_input(tx,0,address.to_script_pub_key())
txin.script_sig = Script([sig, public_key.to_hex()])
signed_tx = tx.serialize()

print("\nSigned transaction:\n", signed_tx)


Raw unsigned transaction:
0200000001018c3d3411d7eab0d511ecfbf392862b5fb115110ecc48897a1bd62154a07fcc0000000000ffffffff01a0860100000000001976a9148b74738efe6d0d27656e584370aaebf13df1d68488ac00000000

Signed transaction:
 0200000001018c3d3411d7eab0d511ecfbf392862b5fb115110ecc48897a1bd62154a07fcc000000006a47304402202160a28e6f879d23a045fad6ab2a4016fc897449a2655b9ca45b050de219e81d02203f1fdcd9de55377a96bdc6ade157d1a509b4bab7f7fe9b05e6becd085db8c8cc0121032f0d9e099d17b67c4bf4aaf4704ab99693177d26c7e4c598705e442836340327ffffffff01a0860100000000001976a9148b74738efe6d0d27656e584370aaebf13df1d68488ac00000000


# Task 2: Multisignature transaction

- The sender's address:
  - Private Key 1: cNQhoYBf1Cz9ajftiNLpG79s6bU6u3Kkr57vKWE389ybZaiS9B8c
  - Private Key 2: cSXUbRHVAKkgF9dtdSUdGQq6rFukRnk5T2Rsgz6uchofy2uyRJYC
  - Redeem Script: 5221038caddaf41718b22f3f3abb87135bad994ed87f68816ad6d62e6a08c85d80ed3c210222a9bdbe0dc6f7ed6f2912f828b73d23edc86c070b3e66fa22cc192d5367942252ae
  - Multisig Address: 2ND1NiSbEeep4cSagVBUb5H7z6D11Rxiyoi

- The recipient's address:
  - Private key: cTcXHR9mpY3Yh1XwQaGrrFRRpYR1XqPGNTb2PjMpPQ84kkafLyRN
  - Public key: 032f0d9e099d17b67c4bf4aaf4704ab99693177d26c7e4c598705e442836340327
  - Bitcoin address: mr3q9zULfWJktFDZMgpkTCa5ra9vJDdcJ4

- The previous transaction identifier (txid): fe69cd5bffdfaed5a969428d774abdd3249c7f27c0ca494e5d511ccd92e19db2

In [4]:
from bitcoinutils.script import Script
from bitcoinutils.keys import P2shAddress, PrivateKey

In [5]:
#setup the network
setup('testnet')

# Generate two random private keys
#private_key1 = PrivateKey.from_bytes(os.urandom(32))
#private_key2 = PrivateKey.from_bytes(os.urandom(32))
private_key1 = PrivateKey.from_wif("cNQhoYBf1Cz9ajftiNLpG79s6bU6u3Kkr57vKWE389ybZaiS9B8c")
private_key2 = PrivateKey.from_wif("cSXUbRHVAKkgF9dtdSUdGQq6rFukRnk5T2Rsgz6uchofy2uyRJYC")

# Derive the public keys
public_key1 = private_key1.get_public_key()
public_key2 = private_key2.get_public_key()

# Create a 2-of-2 multisig redeem script
redeem_script = Script(["OP_2", public_key1.to_hex(), public_key2.to_hex(), "OP_2", "OP_CHECKMULTISIG"])

# Create a P2SH address from the redeem script
address = P2shAddress.from_script(redeem_script)

# print address and hash160
print("Private Key 1:", private_key1.to_wif())
print("Private Key 2:", private_key2.to_wif())
print("Redeem Script:", redeem_script.to_hex())
print("Multisig Address:", address.to_string())

Private Key 1: cNQhoYBf1Cz9ajftiNLpG79s6bU6u3Kkr57vKWE389ybZaiS9B8c
Private Key 2: cSXUbRHVAKkgF9dtdSUdGQq6rFukRnk5T2Rsgz6uchofy2uyRJYC
Redeem Script: 5221038caddaf41718b22f3f3abb87135bad994ed87f68816ad6d62e6a08c85d80ed3c210222a9bdbe0dc6f7ed6f2912f828b73d23edc86c070b3e66fa22cc192d5367942252ae
Multisig Address: 2ND1NiSbEeep4cSagVBUb5H7z6D11Rxiyoi


In [8]:
# create transaction input from txid of UTXO
txin = TxInput(
    "fe69cd5bffdfaed5a969428d774abdd3249c7f27c0ca494e5d511ccd92e19db2", 0
)

# create a transaction output to the generated address
to_addr = PrivateKey.from_wif("cTcXHR9mpY3Yh1XwQaGrrFRRpYR1XqPGNTb2PjMpPQ84kkafLyRN").get_public_key().get_address()
txout = TxOutput(
        to_satoshis(0.001),
        Script(["OP_DUP", "OP_HASH160", to_addr.to_hash160(), "OP_EQUALVERIFY", "OP_CHECKSIG"]),
    )

# create the transaction
tx = Transaction([txin], [txout])
print("\nRaw unsigned transaction:\n" + tx.serialize())

# Sign the transaction
print(private_key1.to_wif())
sig1 = private_key1.sign_input(tx,0,redeem_script)
sig2 = private_key2.sign_input(tx,0,redeem_script)

txin.script_sig = Script(["OP_0", sig1, sig2, redeem_script.to_hex()])
signed_tx = tx.serialize()

print("\nSigned transaction:\n", signed_tx)


Raw unsigned transaction:
0200000001b29de192cd1c515d4e49cac0277f9c24d3bd4a778d4269a9d5aedfff5bcd69fe0000000000ffffffff01a0860100000000001976a9147388317cdde2c6629eb7234e171ac67b5ef4960f88ac00000000
cNQhoYBf1Cz9ajftiNLpG79s6bU6u3Kkr57vKWE389ybZaiS9B8c

Signed transaction:
 0200000001b29de192cd1c515d4e49cac0277f9c24d3bd4a778d4269a9d5aedfff5bcd69fe00000000d900473044022013d51caed0da4cfd481a40d7fb8b585e15844cb169181802b9ea49c1d363680c02207637936ef31f11eae28d2552afd618bc660b81c6b8dca1cdbf244c0f7fe4ad5d0147304402206b057673d4dfeb9a96a710070b889eba319276b2550a4854166fdfb8513930f902205741fc17ab0ec39f8edd85fc9293a7a2c95df8bb1946a158a2b73b4e66107ec101475221038caddaf41718b22f3f3abb87135bad994ed87f68816ad6d62e6a08c85d80ed3c210222a9bdbe0dc6f7ed6f2912f828b73d23edc86c070b3e66fa22cc192d5367942252aeffffffff01a0860100000000001976a9147388317cdde2c6629eb7234e171ac67b5ef4960f88ac00000000
